In [13]:
import os
import cv2
from tqdm import tqdm

# -------- CONFIG --------
DATASET = {
    'train': {
        'img_dir': 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/images',
        'label_dir': 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/labels',
        'output_dir': 'train_output'
    },
    'test': {
        'img_dir': 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/images',
        'label_dir': 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/labels',
        'output_dir': 'test_output'
    }
}

# YOLO class names (edit this according to your dataset)
CLASS_NAMES = {
    0: "spacecraft"
    # Add more classes if needed
}

def draw_boxes(image_path, label_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Failed to load image: {image_path}")
        return None

    h, w = img.shape[:2]

    if not os.path.exists(label_path):
        print(f"⚠️ Label file not found for image: {image_path}")
        return img  # No label file, return original image

    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"⚠️ Malformed line in label file: {label_path}")
                continue
            cls_id, x_c, y_c, box_w, box_h = map(float, parts)

            # Convert to pixel coordinates
            x_c, y_c, box_w, box_h = x_c * w, y_c * h, box_w * w, box_h * h
            x1 = int(x_c - box_w / 2)
            y1 = int(y_c - box_h / 2)
            x2 = int(x_c + box_w / 2)
            y2 = int(y_c + box_h / 2)

            label = CLASS_NAMES.get(int(cls_id), str(int(cls_id)))

            # Draw rectangle and label
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)

    return img

# -------- PROCESS --------
for split in DATASET:
    img_dir = DATASET[split]['img_dir']
    label_dir = DATASET[split]['label_dir']
    output_dir = DATASET[split]['output_dir']
    os.makedirs(output_dir, exist_ok=True)

    image_files = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    print(f"📂 Processing {split} set...")
    print(f"📷 Found {len(image_files)} images in: {img_dir}")

    for img_file in tqdm(image_files, desc=f"Processing {split} set"):
        img_path = os.path.join(img_dir, img_file)
        label_file = os.path.splitext(img_file)[0] + '.txt'
        label_path = os.path.join(label_dir, label_file)

        result_img = draw_boxes(img_path, label_path)
        if result_img is None:
            continue

        output_path = os.path.join(output_dir, img_file)
        success = cv2.imwrite(output_path, result_img)
        if not success:
            print(f"❌ Failed to save output image: {output_path}")

print("✅ Done drawing all bounding boxes and saving images.")


📂 Processing train set...
📷 Found 9952 images in: C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/images


Processing train set: 100%|██████████| 9952/9952 [00:27<00:00, 360.14it/s]


📂 Processing test set...
📷 Found 10793 images in: C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/images


Processing test set: 100%|██████████| 10793/10793 [00:34<00:00, 314.46it/s]

✅ Done drawing all bounding boxes and saving images.


In [15]:
import mediapipe as mp
print(mp.__version__)


0.10.7


In [16]:
import os
from PIL import Image

# Paths
image_dir = "C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/images"
label_dir = "C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/labels"
output_dir = "C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/labels_with_keypoints"

os.makedirs(output_dir, exist_ok=True)

for label_file in os.listdir(label_dir):
    if not label_file.endswith(".txt"):
        continue

    image_name = label_file.replace(".txt", ".jpg")
    image_path = os.path.join(image_dir, image_name)
    label_path = os.path.join(label_dir, label_file)

    if not os.path.exists(image_path):
        continue

    img = Image.open(image_path)
    img_w, img_h = img.size

    with open(label_path, "r") as f:
        lines = f.readlines()

    with open(os.path.join(output_dir, label_file), "w") as out:
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            cls, x_center, y_center, w, h = map(float, parts[:5])
            
            # Convert from normalized to pixel values
            x_c = x_center * img_w
            y_c = y_center * img_h
            bw = w * img_w
            bh = h * img_h

            # Bounding box corners
            x_min = x_c - bw / 2
            x_max = x_c + bw / 2
            y_min = y_c - bh / 2
            y_max = y_c + bh / 2

            # Keypoints (in pixels)
            keypoints = [
                (x_min, y_min),  # Top-left
                (x_max, y_min),  # Top-right
                (x_min, y_max),  # Bottom-left
                (x_max, y_max),  # Bottom-right
                (x_c, y_c)       # Center
            ]

            # Write bbox line
            out.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
            
            # Write keypoints in normalized format
            for idx, (kp_x, kp_y) in enumerate(keypoints):
                kp_x_norm = kp_x / img_w
                kp_y_norm = kp_y / img_h
                out.write(f"{kp_x_norm:.6f} {kp_y_norm:.6f} {float(idx):.1f}\n")


In [17]:
import os
from PIL import Image

# Paths
image_dir = "C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/images"
label_dir = "C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/labels"
output_dir = "C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/labels_with_keypoints"

os.makedirs(output_dir, exist_ok=True)

for label_file in os.listdir(label_dir):
    if not label_file.endswith(".txt"):
        continue

    image_name = label_file.replace(".txt", ".jpg")
    image_path = os.path.join(image_dir, image_name)
    label_path = os.path.join(label_dir, label_file)

    if not os.path.exists(image_path):
        continue

    img = Image.open(image_path)
    img_w, img_h = img.size

    with open(label_path, "r") as f:
        lines = f.readlines()

    with open(os.path.join(output_dir, label_file), "w") as out:
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            cls, x_center, y_center, w, h = map(float, parts[:5])
            
            # Convert from normalized to pixel values
            x_c = x_center * img_w
            y_c = y_center * img_h
            bw = w * img_w
            bh = h * img_h

            # Bounding box corners
            x_min = x_c - bw / 2
            x_max = x_c + bw / 2
            y_min = y_c - bh / 2
            y_max = y_c + bh / 2

            # Keypoints (in pixels)
            keypoints = [
                (x_min, y_min),  # Top-left
                (x_max, y_min),  # Top-right
                (x_min, y_max),  # Bottom-left
                (x_max, y_max),  # Bottom-right
                (x_c, y_c)       # Center
            ]

            # Write bbox line
            out.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
            
            # Write keypoints in normalized format
            for idx, (kp_x, kp_y) in enumerate(keypoints):
                kp_x_norm = kp_x / img_w
                kp_y_norm = kp_y / img_h
                out.write(f"{kp_x_norm:.6f} {kp_y_norm:.6f} {float(idx):.1f}\n")


In [18]:
import os
import cv2

def draw_bbox_and_keypoints(image_path, label_path, save_path):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    with open(label_path, 'r') as f:
        lines = f.read().strip().split('\n')

    if not lines or len(lines) < 6:
        print(f"Skipping: {image_path} due to insufficient keypoints.")
        return

    # === BBOX ===
    class_id, cx, cy, bw, bh = map(float, lines[0].split())
    x1 = int((cx - bw / 2) * w)
    y1 = int((cy - bh / 2) * h)
    x2 = int((cx + bw / 2) * w)
    y2 = int((cy + bh / 2) * h)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # === KEYPOINTS ===
    for line in lines[1:6]:  # 5 keypoints
        x_norm, y_norm, idx = map(float, line.split())
        x = int(x_norm * w)
        y = int(y_norm * h)
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.putText(img, str(int(idx)), (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    cv2.imwrite(save_path, img)

# === CONFIG ===
def process_dataset(img_dir, label_dir, save_dir):
    for file in os.listdir(img_dir):
        if file.endswith(".jpg") or file.endswith(".png"):
            img_path = os.path.join(img_dir, file)
            label_path = os.path.join(label_dir, os.path.splitext(file)[0] + ".txt")
            save_path = os.path.join(save_dir, file)

            if os.path.exists(label_path):
                draw_bbox_and_keypoints(img_path, label_path, save_path)
            else:
                print(f"Label not found for: {file}")

# === PATHS ===
TRAIN_IMG_DIR = 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/images'
TRAIN_LABEL_DIR = 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/labels_with_keypoints'
TRAIN_SAVE_DIR = 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/train_visualised'

TEST_IMG_DIR = 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/images'
TEST_LABEL_DIR = 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/labels_with_keypoints'
TEST_SAVE_DIR = 'C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/test_visualised'

# === RUN ===
process_dataset(TRAIN_IMG_DIR, TRAIN_LABEL_DIR, TRAIN_SAVE_DIR)
process_dataset(TEST_IMG_DIR, TEST_LABEL_DIR, TEST_SAVE_DIR)


In [19]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from scipy.spatial.transform import Rotation as R

# Camera configuration
IMG_WIDTH = 2448
IMG_HEIGHT = 2048
SENSOR_WIDTH_MM = 8.4456
SENSOR_HEIGHT_MM = 7.0656
FOCAL_LENGTH_MM = 12

FOCAL_LENGTH_PIX_X = FOCAL_LENGTH_MM / SENSOR_WIDTH_MM * IMG_WIDTH
FOCAL_LENGTH_PIX_Y = FOCAL_LENGTH_MM / SENSOR_HEIGHT_MM * IMG_HEIGHT
CAMERA_MATRIX = np.array([
    [FOCAL_LENGTH_PIX_X, 0, IMG_WIDTH / 2],
    [0, FOCAL_LENGTH_PIX_Y, IMG_HEIGHT / 2],
    [0, 0, 1]
])
DIST_COEFFS = np.zeros((4, 1))

def parse_label(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()

    bbox = list(map(float, lines[0].strip().split()[1:]))  # skip class_id
    keypoints = []
    for kp_line in lines[1:]:
        parts = kp_line.strip().split()
        if len(parts) == 3:
            x, y = float(parts[0]), float(parts[1])
            keypoints.append([x * IMG_WIDTH, y * IMG_HEIGHT])
    return bbox, np.array(keypoints, dtype=np.float32)

def get_3d_keypoints(n):
    """Assume a square flat target (or known depth). Customize this if needed."""
    size = 0.1  # in meters
    return np.array([[i % 2 * size, i // 2 * size, 0] for i in range(n)], dtype=np.float32)

def estimate_pose(image_path, label_path):
    bbox, img_keypoints = parse_label(label_path)
    if len(img_keypoints) < 4:
        return None

    obj_keypoints = get_3d_keypoints(len(img_keypoints))
    success, rvec, tvec = cv2.solvePnP(obj_keypoints, img_keypoints, CAMERA_MATRIX, DIST_COEFFS)
    if not success:
        return None

    rot_mat, _ = cv2.Rodrigues(rvec)
    r = R.from_matrix(rot_mat)
    quat = r.as_quat()  # x, y, z, w
    euler = r.as_euler('xyz', degrees=True)  # roll, pitch, yaw

    return [os.path.basename(image_path), *tvec.flatten(), *euler, *quat]

def process_dataset(img_dir, label_dir, save_csv):
    results = []
    image_paths = sorted(glob(os.path.join(img_dir, '*.jpg')))

    for img_path in tqdm(image_paths):
        img_name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_dir, f"{img_name}.txt")

        if not os.path.exists(label_path):
            continue

        pose = estimate_pose(img_path, label_path)
        if pose:
            results.append(pose)

    df = pd.DataFrame(results, columns=[
        "IMG_NUM", "X", "Y", "Z", "ROLL", "PITCH", "YAW", "Q1", "Q2", "Q3", "W"
    ])
    df.to_csv(save_csv, index=False)
    print(f"Saved {len(df)} poses to {save_csv}")

# 🧪 PASS YOUR PATHS HERE:
process_dataset("C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/train_visualised", "C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/labels_with_keypoints", "C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/train_pose_results.csv")
process_dataset("C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/test_visualised", "C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/labels_with_keypoints", "C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/test_pose_results.csv")


100%|██████████| 9952/9952 [00:07<00:00, 1420.22it/s]


Saved 9952 poses to C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/train_pose_results.csv


100%|██████████| 10793/10793 [00:07<00:00, 1427.26it/s]


Saved 10793 poses to C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/test_pose_results.csv


In [21]:
import pandas as pd

# Load predicted pose file
pred = pd.read_csv("C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/train_pose_results.csv")

# Clean filenames
pred["IMG_NUM"] = pred["IMG_NUM"].apply(lambda x: x.split("_jpg")[0] + ".jpg")

# Save cleaned version
pred.to_csv("C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/train_pose_results1.csv", index=False)


In [20]:
import pandas as pd

# Load predicted pose file
pred = pd.read_csv("C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/test_pose_results.csv")

# Clean filenames
pred["IMG_NUM"] = pred["IMG_NUM"].apply(lambda x: x.split("_jpg")[0] + ".jpg")

# Save cleaned version
pred.to_csv("C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/test_pose_results1.csv", index=False)


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, f1_score,auc,
    mean_squared_error, mean_absolute_error,
)

# --- Step 1: Load the Data ---
gt = pd.read_csv('C:/Users/rohit/Downloads/archive/synthetic_cubesat/dataset/train/train_ground_truth.csv')
pred = pd.read_csv('C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/train_pose_results1.csv')

# Sort by IMG_NUM for alignment
gt = gt.sort_values(by='IMG_NUM').reset_index(drop=True)
pred = pred.sort_values(by='IMG_NUM').reset_index(drop=True)

assert len(gt) == len(pred), "Mismatch in number of rows between ground truth and predictions"

# --- Step 2: Calculate Errors ---
# Translational Error
trans_error = np.linalg.norm(gt[['X', 'Y', 'Z']].values - pred[['X', 'Y', 'Z']].values, axis=1)

# Rotational Error (Quaternion difference)
def quat_error(q1, q2):
    dot = np.abs(np.sum(q1 * q2, axis=1))  # absolute dot product
    dot = np.clip(dot, -1.0, 1.0)
    return 2 * np.arccos(dot) * 180 / np.pi  # degrees

q_gt = gt[['Q1', 'Q2', 'Q3', 'W']].values
q_pred = pred[['Q1', 'Q2', 'Q3', 'W']].values
rot_error = quat_error(q_gt, q_pred)

# Total Error
total_error = np.sqrt(trans_error**2 + rot_error**2)

# --- Step 3: Apply Threshold ---
threshold = 170.0  # set your threshold here
y_true = np.ones(len(total_error))  # ground truth: always acceptable (1)
y_pred = (total_error <= threshold).astype(int)  # 1 = pass, 0 = fail

# --- Step 4: Save Everything to CSV ---
df_out = pd.DataFrame({
    'IMG_NUM': gt['IMG_NUM'],
    'Translational_Error': trans_error,
    'Rotational_Error': rot_error,
    'Total_Error': total_error,
    'Pass_or_Fail': y_pred
})
df_out.to_csv('C:/Users/rohit/Downloads/archive/synthetic_cubesat/train dataset/train/train_pose_error_with_threshold.csv', index=False)

# --- Step 5: Metrics ---
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# --- Step 6: Regression Metrics ---
mse = mean_squared_error(np.zeros_like(total_error), total_error)
rmse = np.sqrt(mse)
mae = mean_absolute_error(np.zeros_like(total_error), total_error)


# --- Step 7: Plot Metrics and Save ---
# Confusion Matrix


# F1 Score Plot
plt.figure()
plt.bar(['F1 Score'], [f1], color='purple')
plt.ylim(0, 1)
plt.title('F1 Score')
plt.savefig('f1_score.png')
plt.close()

# Accuracy Plot
plt.figure()
plt.bar(['Accuracy'], [accuracy], color='green')
plt.ylim(0, 1)
plt.title('Accuracy')
plt.savefig('accuracy.png')
plt.close()



# --- Step 8: Print Metrics ---
print("📊 Classification Metrics:")
print(f"✅ Accuracy       : {accuracy:.4f}")
print(f"✅ F1 Score       : {f1:.4f}")


print("\n📈 Regression Error Metrics on Total Error:")
print(f"✅ MSE            : {mse:.4f}")
print(f"✅ RMSE           : {rmse:.4f}")
print(f"✅ MAE            : {mae:.4f}")



📊 Classification Metrics:
✅ Accuracy       : 0.9011
✅ F1 Score       : 0.9480

📈 Regression Error Metrics on Total Error:
✅ MSE            : 16997.6785
✅ RMSE           : 130.3751
✅ MAE            : 125.4481


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, f1_score,auc,
    mean_squared_error, mean_absolute_error,
)

# --- Step 1: Load the Data ---
gt = pd.read_csv('C:/Users/rohit/Downloads/archive/synthetic_cubesat/sequence_a/test_ground_truth.csv')
pred = pd.read_csv('C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/test_pose_results1.csv')

# Sort by IMG_NUM for alignment
gt = gt.sort_values(by='IMG_NUM').reset_index(drop=True)
pred = pred.sort_values(by='IMG_NUM').reset_index(drop=True)

assert len(gt) == len(pred), "Mismatch in number of rows between ground truth and predictions"

# --- Step 2: Calculate Errors ---
# Translational Error
trans_error = np.linalg.norm(gt[['X', 'Y', 'Z']].values - pred[['X', 'Y', 'Z']].values, axis=1)

# Rotational Error (Quaternion difference)
def quat_error(q1, q2):
    dot = np.abs(np.sum(q1 * q2, axis=1))  # absolute dot product
    dot = np.clip(dot, -1.0, 1.0)
    return 2 * np.arccos(dot) * 180 / np.pi  # degrees

q_gt = gt[['Q1', 'Q2', 'Q3', 'W']].values
q_pred = pred[['Q1', 'Q2', 'Q3', 'W']].values
rot_error = quat_error(q_gt, q_pred)

# Total Error
total_error = np.sqrt(trans_error**2 + rot_error**2)

# --- Step 3: Apply Threshold ---
threshold = 170.0  # set your threshold here
y_true = np.ones(len(total_error))  # ground truth: always acceptable (1)
y_pred = (total_error <= threshold).astype(int)  # 1 = pass, 0 = fail

# --- Step 4: Save Everything to CSV ---
df_out = pd.DataFrame({
    'IMG_NUM': gt['IMG_NUM'],
    'Translational_Error': trans_error,
    'Rotational_Error': rot_error,
    'Total_Error': total_error,
    'Pass_or_Fail': y_pred
})
df_out.to_csv('C:/Users/rohit/Downloads/archive/synthetic_cubesat/test dataset/test/test_pose_error_with_threshold.csv', index=False)

# --- Step 5: Metrics ---
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# --- Step 6: Regression Metrics ---
mse = mean_squared_error(np.zeros_like(total_error), total_error)
rmse = np.sqrt(mse)
mae = mean_absolute_error(np.zeros_like(total_error), total_error)


# --- Step 7: Plot Metrics and Save ---
# Confusion Matrix


# F1 Score Plot
plt.figure()
plt.bar(['F1 Score'], [f1], color='purple')
plt.ylim(0, 1)
plt.title('F1 Score')
plt.savefig('test_f1_score.png')
plt.close()

# Accuracy Plot
plt.figure()
plt.bar(['Accuracy'], [accuracy], color='green')
plt.ylim(0, 1)
plt.title('Accuracy')
plt.savefig('test_accuracy.png')
plt.close()



# --- Step 8: Print Metrics ---
print("📊 Classification Metrics:")
print(f"✅ Accuracy       : {accuracy:.4f}")
print(f"✅ F1 Score       : {f1:.4f}")


print("\n📈 Regression Error Metrics on Total Error:")
print(f"✅ MSE            : {mse:.4f}")
print(f"✅ RMSE           : {rmse:.4f}")
print(f"✅ MAE            : {mae:.4f}")



📊 Classification Metrics:
✅ Accuracy       : 0.9397
✅ F1 Score       : 0.9689

📈 Regression Error Metrics on Total Error:
✅ MSE            : 13310.8558
✅ RMSE           : 115.3727
✅ MAE            : 109.2404
